In [41]:
import pandas as pd
import random

import torch

import numpy as np
import re
from sklearn.metrics import mean_squared_error

In [42]:
random.seed(42)

In [43]:
output_hn = "./data/bmrb_IDs_monomer_pH68.txt"
feature_file = "./data/bmrb_detail_monomer_pH68.txt"
fasta_file = "./data/bmrb_monomer_pH68.fasta"

cluster_mmseqs2 = "./data/clusterRes_cluster.tsv" #seq_id = 30%
cluster_rep_mmseqs2 = "./data/clusterRes_rep_seq.fasta" #seq_id = 30%

graph_part = "./data/graphpart/graphpart_assignments.csv" #seq_id = 30% test=20% val=10%

In [44]:
def get_cluster_representatives(cluster_rep):
    cl_repr = []

    with open(cluster_rep, 'r') as fasta:
            lines = fasta.read().split("\n")
            for line in lines:
                if line.startswith(">"):
                    cl_repr.append(int(line[1:]))

    print("Cluster Representative Sequences: "+str(len(cl_repr)))
    
    return cl_repr

In [45]:
def read_fasta(fasta):
    fasta_dct = {}

    with open(fasta, 'r') as f:
            lines = f.read().split("\n")
            for i,line in enumerate(lines):
                if line.startswith(">"):
                    fasta_dct[int(line[1:])] = lines[i+1]

    return fasta_dct

In [46]:
def filter_df(df, ids):
    
    df = df.drop(df[~df['BRMB_ID'].isin(ids)].index)
    df = df.reset_index(drop=True)
    
    return df

In [47]:
def get_cluster_members(file):
    
    df_clusters = pd.read_csv(file, sep="\t", header = None)

    #df for cluster members
    cluster_members = {}

    for index, row in df_clusters.iterrows():
        rep = row[0]
        memb = row[1]

        if rep not in cluster_members:
            cluster_members[rep] = [memb]
        else:
            l = cluster_members[rep]
            l.append(memb)
            cluster_members[rep] = l
            
    return cluster_members

In [48]:
cl_repr = get_cluster_representatives(cluster_rep_mmseqs2)

df_features = pd.read_csv(feature_file, sep="\t")
df_features_mmseqs2 = filter_df(df_features, cl_repr)
#df_features_mmseqs2

Cluster Representative Sequences: 3392


In [49]:
fasta_dct = read_fasta(fasta_file)

cluster_members = get_cluster_members(cluster_mmseqs2) 


#add size of clusters to data frame
cl_size = [cluster_members[row[0]] for index, row in df_features_mmseqs2.iterrows()]
df_features_mmseqs2['Cluster_Size'] = cl_size
df_features_mmseqs2

,BRMB_ID,Sequence,Sequence_ID,Ion,pH,Temperature,Experiment,Cluster_Size
0,30079,MTPIEYIDRALALVVDRLARYPGYEVLLSAEKQLQYIRSVLLDRSL...,A,0.10,6.0,298.0,SOLUTION,[30079]
1,11218,GSSGSSGPARPFRVSNHDRSSRRGVMASSLQELISKTLDALVIATG...,A,0.12,7.0,298.0,solution,"[11218, 4574]"
2,7248,RKKKDIRDYNDADMARLLEQWEKDDDIEEGDLPEHKRPSAPIDFSK...,.,0.10,7.0,303.0,NaN,"[7248, 16213]"
3,17664,GPLGSSCKTSWADRVREAAAQRR,.,0.05,6.2,303.0,solution,[17664]
4,5471,MAQTITQSRLRIDANFKRFVDEEVLPGTGLDAAAFWRNFDEIVHDL...,.,0.10,7.1,310.0,NaN,[5471]
...,...,...,...,...,...,...,...,...
3387,50218,RLRATVSRPVSHQRMGTPMVENDSGYKLGQRVRHAKFGEGTIVNME...,.,0.07,6.5,310.0,solution,[50218]
3388,16471,KGIAEKTVLELMNPEAQLPQVYPFAADLYQKELATLQQQSPEHSLT...,.,0.15,7.5,303.0,solution,[16471]
3389,50362,GAMGPKSKEELLREKLSEDQKTHLDWLKEALGNDGEFDKFLGYDES...,.,0.08,6.8,298.0,solution,"[50362, 50360, 50328, 34344]"
3390,27396,SEAVIKVISSACKTYCGKTSPSKKEIGAMLSLLQKEGLLMSPSDLY...,A,0.10,6.0,305.0,solution,"[27396, 30404]"


# Train-Validation-Test Split

In [50]:
def collect_clusters(df, n):
    
    counter = 0
    index_list = []
    ids = []
        
    for index, row in df.iterrows():
        s = len(row[7])
        
        if counter == n:
            break
        else:
            counter += s
            if counter > n:
                counter -= s
                continue
            else:
                index_list.append(index)
                ids.append(row[7])
                
    
    df = df.drop(index_list)
    
    return df, ids

In [51]:
def train_val_test_split(df, test, valid):
    
    # randomly shuffle the dataframe
    df = df.reindex(np.random.permutation(df.index)) 
    
    df_rest, valid = collect_clusters(df, valid)    
    df_rest, test = collect_clusters(df_rest, test)  
    
    train = df_rest['Cluster_Size'].values
    
    

    """ # how many records is one-fifth of the entire dataframe
    #fifth = int(len(df) / 5)

    # Test set (the top fifth from the entire dataframe)
    test_df = df[:test]#[:fifth]

    df_rest = df[test:]#[fifth:]
    #tenth = int(len(df_rest) / 10)

    # Testing set (top half of the remainder two third of the dataframe)
    valid_df = df_rest[:valid]#[:tenth]

    # Validation set (bottom one third)
    train_df = df_rest[valid:]#[tenth:]"""

    return train,valid,test


In [52]:
def write_fasta(df, outfile):

    file = open(outfile, 'w')

    for index, row in df.iterrows():
        id_ = row['BRMB_ID']
        seq = row['Sequence']
        file.write(f'>{id_}\n{seq}\n')

   
    file.close()  

In [63]:
train,valid,test = train_val_test_split(df_features_mmseqs2, test=250, valid=250)


df_train = df_features[df_features['BRMB_ID'].isin(sum(train, []))]
df_test = df_features[df_features['BRMB_ID'].isin(sum(test, []))]
df_valid = df_features[df_features['BRMB_ID'].isin(sum(valid, []))]

write_fasta(df_train, './data/mmseqs2_train_rd.fasta')
write_fasta(df_valid, './data/mmseqs2_valid.fasta')
write_fasta(df_test, './data/mmseqs2_test.fasta')

print(len(df_train))
print(len(df_test))
print(len(df_valid))


6609
250
250


In [55]:
def read_fasta_todict(fasta_file):

    ids = []
    seqs = []

    with open(fasta_file, 'r') as fasta:
            lines = fasta.read().split("\n")
            for line in lines:
                if line.startswith(">"):
                    ids.append(line[1:])
                else:
                    if line != "":
                        seqs.append(line)
                        
    return {"BRMB_ID":ids, "Sequence":seqs}

# Easy_search filtering (mmseqs2)

In [56]:
def red_reduce_es(df, easy_search_file):
    
    df_es = pd.read_csv(easy_search_file, sep="\t", header=None)

    df1 = df_es[df_es[2]> 0.3]
    
    id_delete_1 = set(df1[1].values)
    print("Entries above 0.3: "+str(len(id_delete_1)))

    
    df = df.drop(df[df['BRMB_ID'].isin(id_delete_1)].index)
    
    return df
    

In [57]:
def get_fasta_df(fastafile):
    
    dct = read_fasta_todict(fastafile)
    df = pd.DataFrame.from_dict(dct)
    df = df.astype({'BRMB_ID':'int'})
    
    return df

In [58]:
def redundancy_reduce_fasta(fasta, m8_file, output):
    
    fasta_df = get_fasta_df(fasta)
    #redundancy_reduce fasta
    fasta_new = red_reduce_es(fasta_df,  m8_file)
    write_fasta(fasta_new, output)
   


In [60]:

redundancy_reduce_fasta(fasta="./data/mmseqs2_train_rd.fasta", 
                        m8_file="./data/train_test.m8", 
                        output='./data/mmseqs2_train_rd.fasta')


"""redundancy_reduce_fasta(fasta="./data/graphpart/gp_train.fasta", 
                        m8_file="./data/easy_search/result_gp_test_train.m8", 
                        output='./data/graphpart/gp_train_rd.fasta')"""


Entries above 0.3: 68


'redundancy_reduce_fasta(fasta="./data/graphpart/gp_train.fasta", \n                        m8_file="./data/easy_search/result_gp_test_train.m8", \n                        output=\'./data/graphpart/gp_train_rd.fasta\')'

In [61]:
train = get_fasta_df("./data/mmseqs2_train_rd.fasta")
valid = get_fasta_df("./data/mmseqs2_valid.fasta")
test = get_fasta_df("./data/mmseqs2_test.fasta")

print("Train: "+str(len(train)))
print("Validation: "+str(len(valid)))
print("Test: "+str(len(test)))

#train_seq = train['Sequence'].values
#seqs = [len(x) for x in train_seq]
#plt.hist(seqs, bins=100)
#plt.show()


Train: 6408
Validation: 250
Test: 250


# GraphPart

In [640]:
df_graphpart = pd.read_csv(graph_part, sep=",")


train_gp = df_graphpart[df_graphpart['cluster'] == 1.0]
test_gp = df_graphpart[df_graphpart['cluster'] == 0.0]
valid_gp = df_graphpart[df_graphpart['cluster'] == 2.0]

train_gp = filter_df(df_features, train_gp['AC'])
test_gp = filter_df(df_features, test_gp['AC'])
valid_gp = filter_df(df_features, valid_gp['AC'])


In [132]:
train_gp = get_fasta_df("./data/graphpart/gp_train_rd.fasta")
valid_gp = get_fasta_df("./data/graphpart/gp_valid.fasta")
test_gp = get_fasta_df("./data/graphpart/gp_test.fasta")

print("Train: "+str(len(train_gp)))
print("Validation: "+str(len(valid_gp)))
print("Test: "+str(len(test_gp)))

Train: 9972
Validation: 49
Test: 247


In [642]:
#write_fasta(train_gp, './data/graphpart/gp_train.fasta')
#write_fasta(valid_gp, './data/graphpart/gp_valid.fasta')
#write_fasta(test_gp, './data/graphpart/gp_test.fasta')

# Easy_search Coverage

In [115]:
def fill_dict(peptide, start, end, dictionary):
    
    if peptide not in dictionary:
        dictionary[peptide] = [[start,end]]
    else:
        pos_list = dictionary[peptide]
        l = []
        for pos in pos_list:
            s = pos[0]
            e = pos[1]
            
            if start < s and s < end:
                if end < e:
                    l.append([start,e])
                else:
                    l.append([start,end])
                
            elif s < start and start < e:
                if e < end:
                    l.append([s,end])
                else: 
                    l.append([s,e])
                
            elif s == start and end == e:
                l.append([s,e])
                
            else:
                l.append([s,e])
                l.append([start,end])
                
                
        dictionary[peptide] = l
        
    return dictionary

In [116]:
def get_alignments(df, both):
    
    peptide_cov = {} #{peptide: [start, end]}

    if both:
        for index, row in df.iterrows():
            q = row['query']
            t = row['target']

            peptide_cov = fill_dict(int(row['query']), int(row['qstart']), int(row['qend']), peptide_cov)
            peptide_cov = fill_dict(int(row['target']), int(row['tstart']), int(row['tend']), peptide_cov)  
    else:
        for index, row in df.iterrows():
            q = row['query']
            peptide_cov = fill_dict(int(row['query']), int(row['qstart']), int(row['qend']), peptide_cov)
            

    return peptide_cov

In [468]:
def calculate_coverage(alignment_dict, df_features):


    # calculate peptide coverage
    coverage_dict = {} #{peptide: coverage}
    coverage_list = []

    for entry, algn in alignment_dict.items():
        seq = df_features[df_features["BRMB_ID"]==entry]["Sequence"].values[0]
        seq_len = len(seq)

        algn_len = 0
        for pos in algn:
            algn_len = algn_len + (pos[1]-pos[0]+1)

        #coverage
        cov = algn_len/seq_len
        coverage_dict[entry] = cov
        coverage_list.append(cov)
        #print(str(entry)+": "+str(seq_len)+", "+str(algn_len))

    mean = sum(coverage_list) / len(coverage_list)
    print("Mean of Coverage: "+str(round(mean,2)))


# Easy_search Accuracy

In [107]:
def fill_dict_hn(pos_dict, peptide, position, peptide_dict_h, value_h, peptide_dict_n, value_n, ignore_existing_position):
    
    if peptide not in peptide_dict_h:
        peptide_dict_h[peptide] = [[position, value_h]]
        peptide_dict_n[peptide] = [[position, value_n]]
        pos_dict[peptide] = [position]

    else:
        list_tmp_h = peptide_dict_h[peptide]
        list_tmp_n = peptide_dict_n[peptide]
        list_i = pos_dict[peptide]

        #if position already exists in list, take the one with the highest difference
        if position in list_i:
            
            #if ignore exisiting position: don't do anything
            if not ignore_existing_position:
            
                index = list_i.index(position)
                temp_h = list_tmp_h[index][1]
                temp_n = list_tmp_n[index][1]

                if temp_h < value_h:
                    list_tmp_h[index] = [position, value_h]

                if temp_n < value_n:
                    list_tmp_n[index] = [position, value_n]

        else:
            list_tmp_h.append([position, value_h])
            peptide_dict_h[peptide] = list_tmp_h
            
            list_tmp_n.append([position, value_n])
            peptide_dict_n[peptide] = list_tmp_n

            list_i.append(position)
            pos_dict[peptide] =list_i
            
    return peptide_dict_h, peptide_dict_n, pos_dict

In [108]:
def get_hn_per_protein(hn_file, alignments, df, seq_dict):
    
    df_hn = pd.read_csv(hn_file, sep="\t")
    df_hn = df_hn.drop(df_hn[~df_hn['BRMB_ID'].isin(alignments.keys())].index)

    #dict for query-protein values
    peptide_h = {} #{peptide: [(position, h)]}
    peptide_n = {} #{peptide: [(position, n)]}
    
    #dict for query-protein transferd annotation values
    peptide_transfer_h = {} 
    peptide_transfer_n = {} 
    
    
    pep_pos1 = {} #{peptide: [position]}
    pep_pos2 = {}

    
    for index, row in df.iterrows():
        
        q = row['query'] 
        q_pos = list(range(int(row['qstart']),int(row['qend'])+1))

        t = row['target'] 
        t_pos = list(range(int(row['tstart']),int(row['tend'])+1))
        
        
        df_q = df_hn[df_hn['BRMB_ID']== q]
        df_t = df_hn[df_hn['BRMB_ID']== t]

        df_q = df_q.drop(df_q[~df_q['AA_ID'].isin(q_pos)].index)
        df_t = df_t.drop(df_t[~df_t['AA_ID'].isin(t_pos)].index)  


        # go though aligment: M (match), D (deletion, gap in query), or I (Insertion, gap in target)
        alignment = re.split("(M|D|I)",row['cigar'])[:-1] 


        current_pos_q = q_pos[0]
        current_pos_t = t_pos[0]


        #iterate over local aligment
        for i in range(0, len(alignment), 2):

            action = alignment[i+1]
            step = int(alignment[i])

            positions_q =list(range(current_pos_q, current_pos_q+step))
            positions_t =list(range(current_pos_t, current_pos_t+step))

            if action == 'M':
                df_q_cur = df_q.drop(df_q[~df_q['AA_ID'].isin(positions_q)].index)
                df_t_cur = df_t.drop(df_t[~df_t['AA_ID'].isin(positions_t)].index)

                q_l = df_q_cur['AA_ID'].values
                t_l = df_t_cur['AA_ID'].values

                    

                #check if H/N entries for all the positions are annotated
                #extract H/N values for existing ones
                for i in range(0, step):
                    i_q = positions_q[i]
                    i_t = positions_t[i]


                    
                    if i_q in q_l and i_t in t_l:
                        h_q = df_q_cur[df_q_cur['AA_ID'] == i_q]['H'].values[0]
                        n_q = df_q_cur[df_q_cur['AA_ID'] == i_q]['N'].values[0]

                        h_t = df_t_cur[df_t_cur['AA_ID'] == i_t]['H'].values[0]
                        n_t = df_t_cur[df_t_cur['AA_ID'] == i_t]['N'].values[0]
                        
                        #h_diff = abs(h_q - h_t)
                        #n_diff = abs(n_q - n_t)
                            
                        #fill dictionary with peptide: position, h-difference
                        peptide_transfer_h, peptide_transfer_n, pep_pos1 = fill_dict_hn(pep_pos1, q, i_q, peptide_transfer_h, h_t,
                                                                               peptide_transfer_n, n_t, ignore_existing_position=True)
                        
                        peptide_h, peptide_n, pep_pos2 = fill_dict_hn(pep_pos2, q, i_q, peptide_h, h_q,
                                                                               peptide_n, n_q, ignore_existing_position=True)


                current_pos_t = current_pos_t+step
                current_pos_q = current_pos_q+step


            elif action == 'D':
                current_pos_t = current_pos_t+step

            elif action == 'I':
                current_pos_q = current_pos_q+step
        
        
    return  peptide_transfer_h, peptide_h, peptide_transfer_n, peptide_n  




In [109]:
#go through dictionary with biggest H/N differences and 
#put if H_difference<=0.04 and N_difference<=0.2 == 1 else 0 
#take sum and divide by sequence length
#per protein

def get_accuracy_per_protein(dct, threshold, df_length):
    n = threshold
    experimental_error_proteins = {} #{protein: binary_list of difference of H/N within experimental_error}

    for k,value in dct.items():
        binary_list = [1 if v[1] <= n else 0 for v in value]

        seq = df_length[df_length["BRMB_ID"]==k]["Sequence"].values[0]
        seq_len = len(seq)

        experimental_error_proteins[k] = sum(binary_list)/seq_len

    return experimental_error_proteins
        
    

In [110]:
def accucary_mean(dct):
    v =  dct.values()
    return sum(v)/len(v)

In [111]:
def get_mse_mean(dict_true, dict_transfered):
    mses = []
    for protein in dict_true:
        true = [tup[1] for tup in dict_true[protein]]
        pred = [tup[1] for tup in dict_transfered[protein]]
        mse = mean_squared_error(true, pred)
        mses.append(mse)
        
    return sum(mses)/len(mses)

# Main

In [112]:
def get_coverage_accuracy(file, features, hn):
    #--format-output query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits

    easy_search_file = file
    print(re.split("/",easy_search_file)[-1])

    m8 = pd.read_csv(easy_search_file, sep="\t", header=None, 
                         names=["query","target","fident","alnlen","mismatch","gapopen","qstart","qend","tstart","tend","evalue","bits","cigar"])

    
    #filter out rows with alignment > 0.3
    m8 = m8.drop(m8[m8['fident']> 0.3].index)
    #m8 = m8.sort_values(by = 'evalue')
    print(m8)
    
    
    df_features = pd.read_csv(features, sep="\t")
    
    
    alignments_all = get_alignments(m8, both=True)
    alignments_q = get_alignments(m8, both=False)

    calculate_coverage(alignments_q, df_features)


    ####Accuracy
    diffs_h, diffs_n = get_hn_differences_per_protein(hn, alignments_all, m8)

    accuracy_h = get_accuracy_per_protein(diffs_h, 0.04, df_features)
    accuracy_n = get_accuracy_per_protein(diffs_n, 0.2, df_features)

    mean_h = accucary_mean(accuracy_h)
    mean_n = accucary_mean(accuracy_n)

    print("Accuracy H: "+str(round(mean_h,2)))
    print("Accuracy N: "+str(round(mean_n,2)))

In [118]:
def get_coverage_mse(file, features, hn):
    #--format-output query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits

    easy_search_file = file
    print(re.split("/",easy_search_file)[-1])

    m8 = pd.read_csv(easy_search_file, sep="\t", header=None, 
                         names=["query","target","fident","alnlen","mismatch","gapopen","qstart","qend","tstart","tend","evalue","bits","cigar"])

    
    #filter out rows with alignment > 0.3
    m8 = m8.drop(m8[m8['fident']> 0.3].index)
    #print(m8)

    alignments_all = get_alignments(m8, both=True)
    alignments_q = get_alignments(m8, both=False)
    
    df_features = pd.read_csv(features, sep="\t")
    
    #dictionary of query-protein:sequence
    df_seq = df_features[df_features["BRMB_ID"].isin(m8['query'])]
    seq_dict = dict([(i,x) for i,x in zip(df_seq['BRMB_ID'], df_seq['Sequence'])])


    peptide_transfer_h, peptide_h, peptide_transfer_n, peptide_n  = get_hn_per_protein(hn, alignments_all, m8, seq_dict)
    
    #### MSE
    mse_h = get_mse_mean(peptide_h, peptide_transfer_h)
    mse_n = get_mse_mean(peptide_n, peptide_transfer_n)
   
    print("MSE H: "+str(round(mse_h,2)))
    print("MSE N: "+str(round(mse_n,2)))
    
    #### Coverage
    cov_list = []
    for protein in peptide_transfer_h:
        l = len(peptide_transfer_h[protein])
        cov = l/len(seq_dict[protein])
        #print(protein)
        #print("residues: "+str(l)+" of "+str(len(seq_dict[protein])))
        cov_list.append(cov)
   

    coverage = sum(cov_list)/len(cov_list)
    print("Coverage: "+str(round(coverage,2)))
        

In [ ]:
import os

for subdir, dirs, files in os.walk("./data/mmseqs2/"):
    for file in files:
        if file.endswith(".m8"):
            f = os.path.join(subdir, file)
            
            get_coverage_mse(f,feature_file, output_hn)
            print("\n")


#get_coverage_accuracy("./data/easy_search/result_gp_train_valid2.m8")

result_mmseq_valid_train.m8
MSE H: 0.62
MSE N: 36.08
Coverage: 0.65


result_mmseq_test_train.m8


In [491]:
#redundancy_reduce_fasta("./data/mmseqs2/mmseqs2_valid_rd.fasta", "./data/easy_search/result_mmseq_train_valid_rd.m8", "./data/mmseqs2/mmseqs2_valid_rd.fasta")

#Aber man will ja eine möglichst niedrige Coverage 
#und einen möglichst hohen/schlechten MSE bei unserem aktuellen Experiment
#Die Idee ist ja zu schauen inwiefern HBI (homology-based inference) selbst nach redundancy reduction 
#noch funktioniert. Wenn die redundancy reduction perfekt funktioniert hätte, 
#sollte man ja eine coverage von 0 erhalten (absolut keine Hits mehr) und einen MSE der Random entspricht.

Entries above 0.3: 5
